In [42]:
import os

In [43]:
%pwd

'c:\\Users\\havil\\OneDrive\\Desktop\\projects'

In [44]:
os.chdir("../")

In [45]:
%pwd

'c:\\Users\\havil\\OneDrive\\Desktop'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]  = "https://dagshub.com/havilah.b21/End-to-End-Wine-Quality-Prediction-MLOps-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "havilah.b21"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "295f440ab22972bf8ce4531a516301d56685503e"

In [47]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [58]:
from src.MLOps_Wine_Quality_Predicition_project.constants import *
from src.MLOps_Wine_Quality_Predicition_project.utils.common import read_yaml, create_directories , save_json
from src.MLOps_Wine_Quality_Predicition_project.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.MLOps_Wine_Quality_Predicition_project import logger


In [54]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/havilah.b21/End-to-End-Wine-Quality-Prediction-MLOps-Project.mlflow",
           
        )

        return model_evaluation_config


In [59]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [60]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [61]:
try:
    config = ConfigurationManager()
    model_evalution_config = config.get_model_evaluation_config()
    model_evalution_config  = ModelEvaluation( config = model_evalution_config)
    model_evalution_config.log_into_mlflow()
    
except Exception as e:
    raise e 

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\havil\OneDrive\Desktop\projects\End-to-End-Wine-Quality-Prediction-MLOps-Project\MLOps_Wine_Quality_Predicition_project\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\havil\AppData\Local\Temp\ipykernel_11924\143075981.py", line 8, in <module>
    raise e
  File "C:\Users\havil\AppData\Local\Temp\ipykernel_11924\143075981.py", line 2, in <module>
    config = ConfigurationManager()
             ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\havil\AppData\Local\Temp\ipykernel_11924\4036097930.py", line 8, in __init__
    self.config = read_yaml(config_filepath)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\havil\OneDrive\Desktop\projects\End-to-End-Wine-Quality-Prediction-MLOps-Project\MLOps_Wine_Quality_Predicition_project\Lib\site-packages\ensure\main.py", line 872, in __call__
    return_val = self.f(*args, **kwargs)
         

In [64]:
from dataclasses import dataclass
from pathlib import Path
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import yaml
import json

# Utility functions
def read_yaml(path_to_yaml: Path) -> dict:
    with open(path_to_yaml, 'r') as yaml_file:
        return yaml.safe_load(yaml_file)

def create_directories(paths: list):
    for path in paths:
        os.makedirs(path, exist_ok=True)

def save_json(path: Path, data: dict):
    with open(path, "w") as f:
        json.dump(data, f, indent=4)

# Constants (update these paths to your project structure)
CONFIG_FILE_PATH = Path(r"C:\Users\havil\OneDrive\Desktop\projects\End-to-End-Wine-Quality-Prediction-MLOps-Project\config\config.yaml")
PARAMS_FILE_PATH = Path(r"C:\Users\havil\OneDrive\Desktop\projects\End-to-End-Wine-Quality-Prediction-MLOps-Project\params.yaml")
SCHEMA_FILE_PATH = Path(r"C:\Users\havil\OneDrive\Desktop\projects\End-to-End-Wine-Quality-Prediction-MLOps-Project\schema.yaml")


# Configuration dataclass
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    experiment_name: str


# Configuration Manager
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config['artifacts_root']])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config['model_evaluation']
        params = self.params['ElasticNet']
        schema = self.schema['TARGET_COLUMN']

        create_directories([config['root_dir']])

        return ModelEvaluationConfig(
            root_dir=Path(config['root_dir']),
            test_data_path=Path(config['test_data_path']),
            model_path=Path(config['model_path']),
            all_params=params,
            metric_file_name=Path(config['metric_file_name']),
            target_column=schema['name'],
            mlflow_uri="https://dagshub.com/havilah.b21/End-to-End-Wine-Quality-Prediction-MLOps-Project.mlflow",
            experiment_name="wine-quality-prediction"
        )


# Model Evaluation Class
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Set MLflow tracking and experiment
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment(self.config.experiment_name)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=self.config.metric_file_name, data=scores)

            # Log to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

            # Log and (if possible) register the model
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


# Execution
if __name__ == "__main__":
    try:
        config_manager = ConfigurationManager()
        model_eval_config = config_manager.get_model_evaluation_config()
        evaluator = ModelEvaluation(config=model_eval_config)
        evaluator.log_into_mlflow()
    except Exception as e:
        print(f"❌ Error during evaluation: {e}")
        raise e


❌ Error during evaluation: [Errno 2] No such file or directory: 'artifacts\\data_transformation\\test.csv'
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\havil\OneDrive\Desktop\projects\End-to-End-Wine-Quality-Prediction-MLOps-Project\MLOps_Wine_Quality_Predicition_project\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\havil\AppData\Local\Temp\ipykernel_11924\543390064.py", line 130, in <module>
    raise e
  File "C:\Users\havil\AppData\Local\Temp\ipykernel_11924\543390064.py", line 127, in <module>
    evaluator.log_into_mlflow()
  File "C:\Users\havil\AppData\Local\Temp\ipykernel_11924\543390064.py", line 90, in log_into_mlflow
    test_data = pd.read_csv(self.config.test_data_path)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\havil\OneDrive\Desktop\projects\End-to-End-Wine-Quality-Prediction-MLOps-Project\MLOps_Wine_Quality_Predicition_project\Lib\site-packages\pandas\io\parsers\readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)

In [32]:
import mlflow
import logging

# Increase logging level to capture detailed MLflow debug output.
logging.getLogger("mlflow").setLevel(logging.DEBUG)

# Optionally set the tracking URI to point to your MLflow server.
# Replace 'http://localhost:5000' with your actual MLflow tracking server URI if different.
mlflow.set_tracking_uri("https://dagshub.com/havilah.b21/End-to-End-Wine-Quality-Prediction-MLOps-Project.mlflow")

# Set the experiment. This will create the experiment if it does not already exist.
mlflow.set_experiment("test_experiment")

try:
    # Start an MLflow run
    with mlflow.start_run():
        # Log a sample parameter.
        mlflow.log_param("test_param", "test_value")
        print("Run started and parameter logged successfully.")
except Exception as e:
    print("Error while starting MLflow run:", e)


Run started and parameter logged successfully.
🏃 View run merciful-hare-328 at: https://dagshub.com/havilah.b21/End-to-End-Wine-Quality-Prediction-MLOps-Project.mlflow/#/experiments/0/runs/adb6452d90fd40158fa9bc425c095292
🧪 View experiment at: https://dagshub.com/havilah.b21/End-to-End-Wine-Quality-Prediction-MLOps-Project.mlflow/#/experiments/0
